In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import psycopg2
import streamlit as st

In [2]:
#Connect to local postgres DB
conn = psycopg2.connect(**st.secrets["postgres"])
cur = conn.cursor()

In [3]:
def run_query(query_path):
    with open(query_path, 'r') as file:
        query = file.read()
    with conn.cursor() as cur:
        cur.execute(query)
        results = cur.fetchall()
        columns = [desc[0] for desc in cur.description]
        return pd.DataFrame(results, columns=columns)

## Questions To Answer:

- What is the proportion between equity and fixed income over time?
- What is the proportion between sectors over time?
- What is the proportion between regions over time?
- What are the equity inflows over time? Assessing ownership %
- What sectors has the fund been a buyer in?
- What regions/countries has the fund been buying

### Equity and Fixed Income Proportions Over Time

In [4]:
equity_fi_df = run_query('SQL/eq_fi_proportions.sql')
equity_fi_df

,year,Equity Proportion,Fixed Income Proportion
0,1998,39.66,60.34
1,1999,41.87,58.13
2,2000,38.13,61.87
3,2001,40.78,59.22
4,2002,36.74,63.26
5,2003,42.91,57.09
6,2004,39.61,60.39
7,2005,46.47,53.53
8,2006,38.18,61.82
9,2007,45.81,54.19


### Sector Proportions Over Time

In [5]:
sector_prop_df = run_query('SQL/sector_proportions.sql')
sector_prop_df

,year,category,Sector,Proportion of Fund
0,1998,Equity,Basic Materials,1.67
1,1998,Equity,Consumer Discretionary,4.69
2,1998,Equity,Consumer Staples,4.68
3,1998,Equity,Energy,2.23
4,1998,Equity,Financials,9.49
...,...,...,...,...
370,2022,Equity,Utilities,1.87
371,2022,Fixed Income,Corporate Bonds,7.24
372,2022,Fixed Income,Government Bonds,3.07
373,2022,Fixed Income,Securitized Bonds,1.27


### Region Proportions Over Time

In [6]:
region_prop_df = run_query('SQL/region_proportions.sql')
region_prop_df

,year,region,proportion
0,1998,Asia,18.77
1,1998,Europe,50.76
2,1998,International,2.63
3,1998,Latin America,0.10
4,1998,Middle East,0.03
...,...,...,...
186,2022,International,0.73
187,2022,Latin America,0.99
188,2022,Middle East,0.44
189,2022,North America,46.06


### Ownership By Sector Over Time

In [7]:
industry_ownership_df = run_query('SQL/ownership_sector.sql')
industry_ownership_df

,year,Sector,avg_percent_ownership
0,1998,Basic Materials,0.05
1,1998,Consumer Discretionary,0.05
2,1998,Consumer Staples,0.05
3,1998,Energy,0.04
4,1998,Financials,0.05
...,...,...,...
269,2022,Industrials,1.11
270,2022,Real Estate,1.33
271,2022,Technology,1.05
272,2022,Telecommunications,1.01


### Cumulative Change In Percent Ownership By Sector - Last 10 Years

In [8]:
ownership_change_industry_ten_df = run_query('SQL/ownership_change_sector_ten_years.sql')
ownership_change_industry_ten_df

,Sector,cumulative_bp_change_of_ownership
0,Real Estate,0.30
1,Telecommunications,0.17
2,Basic Materials,0.15
3,Industrials,0.15
4,Health Care,0.14
5,Technology,0.14
6,Financials,0.12
7,Consumer Discretionary,0.09
8,Utilities,0.04
9,Consumer Staples,0.03


### Cumulative Change In Percent Ownership By Region - Last 10 Years

In [9]:
ownership_change_region_ten_df = run_query('SQL/ownership_change_region_ten_years.sql')
ownership_change_region_ten_df

,Region,cumulative_bp_change_of_ownership
0,Oceania,0.55
1,Africa,0.48
2,Middle East,0.46
3,Latin America,0.25
4,Asia,0.20
5,North America,0.13
6,Europe,-0.10


### Dynamic Queries - Testing Zone

In [10]:
#Dynamic query function for sector

def run_query_dynamic(query_path, num_years):
    with open(query_path, 'r') as file:
        query = file.read()
    with conn.cursor() as cur:
        cur.execute(query, (num_years,))
        results = cur.fetchall()
        columns = [desc[0] for desc in cur.description]
        return pd.DataFrame(results, columns=columns)

### Sector Dynamic

In [11]:
ownership_change_sector_dynamic_df = run_query_dynamic('SQL/dynamic/ownership_change_sector.sql',3)
ownership_change_sector_dynamic_df

,Sector,cumulative_bp_change_of_ownership
0,Real Estate,-0.06
1,Utilities,-0.07
2,Telecommunications,-0.08
3,Health Care,-0.12
4,Financials,-0.12
5,Technology,-0.13
6,Basic Materials,-0.17
7,Industrials,-0.17
8,Energy,-0.17
9,Consumer Staples,-0.20


### Region Dynamic

In [12]:
ownership_change_region_dynamic_df = run_query_dynamic('SQL/dynamic/ownership_change_region.sql',7)
ownership_change_region_dynamic_df

,Region,cumulative_percent_change_of_ownership
0,Oceania,0.29
1,Europe,0.21
2,Middle East,0.18
3,Latin America,0.16
4,Africa,0.04
5,North America,0.03
6,Asia,-0.11


### Top 10 Companies Dynamic

In [13]:
ownership_change_company_dynamic_df = run_query_dynamic('SQL/dynamic/top10_ownership_change_company.sql',1)
ownership_change_company_dynamic_df

,Company,Sector,cumulative_bp_change_of_ownership
0,Neogen Corp,Health Care,7.72
1,Rogers Corp,Technology,6.81
2,Grainger PLC,Real Estate,5.22
3,Rayonier Inc,Real Estate,4.63
4,Minkabu The Infonoid Inc,Industrials,4.48
5,Dayang Enterprise Holdings Bhd,Energy,3.88
6,Delek US Holdings Inc,Energy,3.87
7,Nissei ASB Machine Co Ltd,Industrials,3.73
8,Ibersol SGPS SA,Consumer Discretionary,3.56
9,Vonovia SE,Real Estate,3.48


### Top 10 Countries Dynamic

In [14]:
ownership_change_country_dynamic_df = run_query_dynamic('SQL/dynamic/top10_ownership_change_country.sql',1)
ownership_change_country_dynamic_df

,Country,cumulative_bp_change_of_ownership
0,Germany,0.35
1,Portugal,0.33
2,Kuwait,0.32
3,Vietnam,0.27
4,Brazil,0.26
5,Sweden,0.24
6,Belgium,0.23
7,Bangladesh,0.23
8,Italy,0.22
9,Netherlands,0.21


### Country Dynamic - Multiselect

In [15]:
def run_query_dynamic_country(query_path, num_years, countries):
    with open(query_path, 'r') as file:
        query = file.read()
    placeholders = ','.join(['%s'] * len(countries))
    formatted_query = query.format(placeholders)
    with conn.cursor() as cur:
        cur.execute(formatted_query, (num_years, *countries))
        results = cur.fetchall()
        columns = [desc[0] for desc in cur.description]
        return pd.DataFrame(results, columns=columns)


In [16]:
ownership_change_country_dynamic_df = run_query_dynamic_country('SQL/dynamic/ownership_change_country_multi_select.sql', 5, ['United States', 'Canada', 'Mexico', 'United Kingdom'])
ownership_change_country_dynamic_df


,Country,cumulative_percent_change_of_ownership
0,Mexico,0.45
1,United Kingdom,0.14
2,United States,-0.17
3,Canada,-0.18
